In [16]:

%pip install folium
%pip install geocoder

from collections import defaultdict
from geopy import distance
import numpy as np
import pandas as pd
import requests
import json
import folium
import json
from folium import plugins
import geocoder


[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
g = geocoder.ip('me')
print(g.latlng)

[40.444, -79.9552]


In [18]:
df = pd.read_csv("data/rapid_data.csv")
addresses = df.loc[:, "streetAddress"]
lat = df.loc[:, "latitude"]
longi = df.loc[:, "longitude"]
cmu_address = (40.4433, -79.944023)
distance_df = pd.DataFrame(columns = ['address1', 'latitude', 'longitude'])
distance_df['price'] = df.loc[:, "price"]
distance_df['address1'] = addresses
distance_df['latitude'] = lat
distance_df['longitude'] = longi
walking = []
driving = []
for i in range(len(addresses)):
  d = distance.distance((distance_df.loc[i, "latitude"], distance_df.loc[i, "longitude"]), (cmu_address[0],cmu_address[1]))
  walking.append(d.miles)
  r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{distance_df.loc[i, "longitude"]},{distance_df.loc[i, "latitude"]};{cmu_address[1]},{cmu_address[0]}?overview=false""")
  route_1 = json.loads(r.content)["routes"][0]
  driving.append(route_1["distance"]*0.000621371)
distance_df["walking"] = walking
distance_df["driving"] = driving
print(distance_df)

                         address1   latitude  longitude  price   walking  \
0               2353 Sidney St #2  40.429380 -79.971140   2700  1.722332   
1              441 Edgemont St #2  40.422600 -79.998880   1200  3.225751   
2                   200 S 15th St  40.424866 -79.983440   1500  2.436547   
3                 5317 Forbes Ave  40.440155 -79.934940   3760  0.525712   
4              4711 Maripoe St #1  40.455498 -79.949670    850  0.892737   
5                   1625 Hybla St  40.474262 -80.037970   1100  5.392646   
6             3202 Hazelhurst Ave  40.383247 -79.965300   1290  4.292837   
7                4245 Parkman Ave  40.448345 -79.957280   4250  0.780735   
8              1007 Galveston Ave  40.453010 -80.017100   1500  3.909921   
9                 11370 Althea Rd  40.468670 -79.832700   1400  6.123022   
10              333 Lehigh Ave #2  40.456150 -79.926100   2995  1.295638   
11  1465 Old Steubenville Pike #B  40.450570 -80.114400   1250  8.995203   
12          

In [31]:
df = distance_df
with open('data/pittsburgh.geojson') as f:
    pittsburgh = json.load(f)
pitt_map = folium.Map(location=[40.4433, -79.944023], tiles='Stamen Toner', zoom_start= 15, tooltip = 'This tooltip will appear on hover')
#add the shape of LA County to the map
# star marker
icon_star = folium.Icon(
    prefix='fa', 
    icon='star', 
    icon_color='yellow',
)
folium.GeoJson(pittsburgh).add_to(pitt_map)
folium.Marker([40.4433, -79.944023], tooltip='Carnegie Mellon University', icon=icon_star).add_to(pitt_map)
for i,row in df.iterrows():
    folium.CircleMarker((row.latitude,row.longitude), radius=5, weight=3, color='red', fill_color='red', fill_opacity=.5, tooltip= f'{round(row.driving, 2)}mi \n ${row.price}/month', popup='<a href=Off_campus.html target=_top> See more </a>',).add_to(pitt_map)
df = pd.read_csv("Data/CMU_listing_data.csv")
for i,row in df.iterrows():
    folium.CircleMarker((row.Latitude,row.Longitude), radius=5, weight=3, color='green', fill_color='green', fill_opacity=.5, tooltip= f'{row.Building} \n ${row.Cost}/semester', popup=f'<a href= https://eringgao.github.io/lucy_listings/Off_campus.html#{row.Cost} target=_top> See more </a>', target='_top').add_to(pitt_map)
pitt_map.save('pitt_map.html')
